# Dummy Example

This notebook shows the use of the dummy feedback function provider which
behaves like the huggingface provider except it does not actually perform any
network calls and just produces constant results. It can be used to prototype
feedback function wiring for your apps before invoking potentially slow (to
run/to load) feedback functions.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.resolve()))

In [ ]:
import threading

from examples.frameworks.custom.custom_app import CustomApp

from trulens_eval import Feedback
from trulens_eval import FeedbackMode
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.feedback.provider.hugs import Huggingface
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.resolve()
)

In [ ]:
# hugs = Huggingface()
hugs = Dummy()

f_positive_sentiment = Feedback(hugs.positive_sentiment).on_output()

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # feedback_mode=FeedbackMode.WITH_APP
    feedbacks=[f_positive_sentiment]
)

In [ ]:
# Test of a sequence of records:
"""
with ta:
    for i, q in enumerate(["hello there"] * 100):
        print(f"\rrequest {i} ", end="")
        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")

        res = ca.respond_to_query(input=q)
"""

In [ ]:
from ipywidgets import interact
from ipywidgets import widgets
from IPython.display import JSON
from trulens_eval.utils.serial import JSONPath

class RecordWiget():
    def __init__(self, record_selections, record=None):
        self.record = record
        self.record_selections=record_selections

        self.d_human = widgets.Output()
        self.d_comp = widgets.Output()
        self.d_extras = widgets.Output()

        self.d = widgets.Output()

        self.human = ""
        self.comp = ""

        self.draw_main()
        self.set_human("...")
        self.draw_selections()

    def draw_selections(self):
        if self.record is None:
            return

        self.d_extras.clear_output(wait=True)

        for s in self.record_selections:
            q = JSONPath.of_string(s)
            try:
                v = str(next(iter(q(self.record.dict()))))[0:16]
            except Exception as e:
                v = str(e)

            with self.d_extras:
                display(widgets.HTML(f"<div><b>{q}</b>: {v}</div>"))

    def set_human(self, human: str):
        self.human = human

        self.d_human.clear_output(wait=True)
        with self.d_human:
            display(
               widgets.HTML(f"<div><b>human:</b> {human}</div>")
            )

    def set_comp(self, comp: str):
        self.comp = comp

        self.d_comp.clear_output(wait=True)
        with self.d_comp:
            display(
               widgets.HTML(f"<div><b>human:</b> {comp}</div>")
            )

    def draw_main(self):
        self.d.clear_output(wait=True)
        with self.d:
            display(widgets.VBox([self.d_human, self.d_comp, self.d_extras]))     

In [ ]:
main_input = widgets.Text(continuous_update=False)

app_selector = widgets.Text(continuous_update=False)
record_selector = widgets.Text(continuous_update=False)

display_bottom = widgets.Output()
display_top = widgets.Output()
display_side = widgets.Output()
display_middle = widgets.Output()

display_records = []

with display_bottom:
    display(main_input)
    display(app_selector)
    display(record_selector)

app_selections = []
record_selections = []

current_record = RecordWiget(record_selections=record_selections)
records = [current_record]

with display_top:
    display(current_record.d)

def update_record_selections():
    for r in records:
        r.draw_selections()

def update_app_selections():
    display_side.clear_output(wait=True)

    for s in app_selections:
        q = JSONPath.of_string(s)

        try:
            v = str(next(iter(q(ta.dict()))))[0:16]
        except Exception as e:
            v = str(e)

        with display_side:
            display(widgets.HTML(f"<div><b>{q}</b><br>{v}</div>"))

def add_app_selection(ev):
    s = ev.new

    if len(s) == 0:
        return

    if s not in app_selections:
        app_selections.append(s)

    update_app_selections()

def add_record_selection(ev):
    s = ev.new

    if len(s) == 0:
        return

    if s not in record_selections:
        record_selections.append(s)

    update_record_selections()

def runit(ev):
    global current_record
    
    human = ev.new

    current_record.set_human(human)

    with ta as recording:
        comp = ca.respond_to_query(input=human)

    current_record.set_comp(comp)

    current_record.record = recording.get()

    current_record.draw_selections()

    update_app_selections()

    current_record = RecordWiget(record_selections=record_selections)
    records.append(current_record)

    with display_top:
        display(current_record.d)


main_input.observe(runit, ["value"])
app_selector.observe(add_app_selection, ["value"])
record_selector.observe(add_record_selection, ["value"])

display(
    widgets.HBox([
        widgets.VBox([
            display_top,
            widgets.HBox([
              widgets.Label("main_input"),
              display_bottom
            ])
        ]),
        display_middle,
        display_side])
    )
